In [1]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.6 MB/s eta 0:00:00


In [2]:
!yolo --version


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
WARNING ⚠️ argument '--version' does not require leading dashes '--', updating to 'version'.
8.3.189


In [3]:
from google.colab import files
uploaded = files.upload()


Saving ToothNumber_TaskDataset.zip to ToothNumber_TaskDataset.zip


In [4]:
!unzip -q /content/ToothNumber_TaskDataset.zip -d /content/ToothNumber_TaskDataset


In [5]:
import os, random, shutil
from glob import glob


dataset_path = "/content/ToothNumber_TaskDataset"
images_path = os.path.join(dataset_path, "images")
labels_path = os.path.join(dataset_path, "labels")


image_files = glob(os.path.join(images_path, "*.jpg")) + glob(os.path.join(images_path, "*.png"))
print(f"Total images found: {len(image_files)}")


random.shuffle(image_files)


n_total = len(image_files)
n_train = int(0.8 * n_total)
n_val = int(0.1 * n_total)
n_test = n_total - n_train - n_val

train_files = image_files[:n_train]
val_files = image_files[n_train:n_train+n_val]
test_files = image_files[n_train+n_val:]

print(f"Train: {len(train_files)}, Val: {len(val_files)}, Test: {len(test_files)}")


for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(images_path, split), exist_ok=True)
    os.makedirs(os.path.join(labels_path, split), exist_ok=True)


def move_files(file_list, split):
    for img_file in file_list:
        file_name = os.path.basename(img_file)
        label_file = os.path.join(labels_path, file_name.rsplit(".", 1)[0] + ".txt")


        shutil.move(img_file, os.path.join(images_path, split, file_name))


        if os.path.exists(label_file):
            shutil.move(label_file, os.path.join(labels_path, split, os.path.basename(label_file)))


move_files(train_files, "train")
move_files(val_files, "val")
move_files(test_files, "test")

print("Dataset successfully split into Train/Val/Test (80/10/10)")


Total images found: 497
Train: 397, Val: 49, Test: 51
Dataset successfully split into Train/Val/Test (80/10/10)


In [6]:
import os
from glob import glob

labels_dir = "/content/ToothNumber_TaskDataset/labels"
label_files = glob(os.path.join(labels_dir, "**/*.txt"), recursive=True)

classes = set()
for file in label_files:
    with open(file, "r") as f:
        for line in f:
            if line.strip():
                cls_id = int(line.split()[0])
                classes.add(cls_id)

classes = sorted(list(classes))
print(" Class IDs found:", classes)
print(" Total classes (nc):", len(classes))


 Class IDs found: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
 Total classes (nc): 32


In [7]:
yaml_text = """
# Tooth Numbering Detection Dataset (FDI System)

path: /content/ToothNumber_TaskDataset   # dataset root folder

train: images/train
val: images/val
test: images/test

nc: 32   # number of classes

names:
  0: Canine (13)
  1: Canine (23)
  2: Canine (33)
  3: Canine (43)
  4: Central Incisor (21)
  5: Central Incisor (41)
  6: Central Incisor (31)
  7: Central Incisor (11)
  8: First Molar (16)
  9: First Molar (26)
  10: First Molar (36)
  11: First Molar (46)
  12: First Premolar (14)
  13: First Premolar (34)
  14: First Premolar (44)
  15: First Premolar (24)
  16: Lateral Incisor (22)
  17: Lateral Incisor (32)
  18: Lateral Incisor (42)
  19: Lateral Incisor (12)
  20: Second Molar (17)
  21: Second Molar (27)
  22: Second Molar (37)
  23: Second Molar (47)
  24: Second Premolar (15)
  25: Second Premolar (25)
  26: Second Premolar (35)
  27: Second Premolar (45)
  28: Third Molar (18)
  29: Third Molar (28)
  30: Third Molar (38)
  31: Third Molar (48)
"""

with open("/content/tooth_data.yaml", "w") as f:
    f.write(yaml_text)
print("data.yaml file created at /content/tooth_data.yaml")


data.yaml file created at /content/tooth_data.yaml


In [8]:
!yolo detect train data=/content/tooth_data.yaml model=yolo11m.pt epochs=100 imgsz=640 batch=16 device=0


Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/tooth_data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True,

In [9]:
!yolo detect val model=runs/detect/train/weights/best.pt data=/content/tooth_data.yaml plots=True


Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,054,704 parameters, 0 gradients, 67.8 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2308.5±421.6 MB/s, size: 71.3 KB)
val: Scanning /content/ToothNumber_TaskDataset/labels/val.cache... 49 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 49/49 974032.7it/s 0.0s
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 33, len(boxes) = 1407. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 0.85it/s 4.7s
                   all         49       1407      0.899       0.91      0.916      0.647
           Canine (13)         47         47      0.927      0.979      0.948      0.655
           Canine (23)    

In [10]:
!yolo detect val model=runs/detect/train/weights/best.pt data=/content/tooth_data.yaml split=val plots=True


Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,054,704 parameters, 0 gradients, 67.8 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2489.5±962.6 MB/s, size: 74.3 KB)
val: Scanning /content/ToothNumber_TaskDataset/labels/val.cache... 49 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 49/49 1053953.3it/s 0.0s
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 33, len(boxes) = 1407. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 0.72it/s 5.6s
                   all         49       1407      0.899       0.91      0.916      0.647
           Canine (13)         47         47      0.927      0.979      0.948      0.655
           Canine (23)   

In [11]:
!yolo detect val model=runs/detect/train/weights/best.pt data=/content/tooth_data.yaml split=test plots=True


Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,054,704 parameters, 0 gradients, 67.8 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 15.9±7.5 MB/s, size: 75.9 KB)
val: Scanning /content/ToothNumber_TaskDataset/labels/test... 51 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 51/51 241.9it/s 0.2s
val: New cache created: /content/ToothNumber_TaskDataset/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 0.67it/s 5.9s
                   all         51       1452      0.926      0.919      0.954      0.711
           Canine (13)         48         48      0.928      0.896      0.963      0.712
           Canine (23)         47         47      0.956      0.926       0.98      0.689
           Canine (33)         51         51       0.92      0.901      0.939      0.699
           Canine (43)         51         52       0.92  

In [13]:
!yolo predict model=runs/detect/train/weights/best.pt source=/content/ToothNumber_TaskDataset/images/test save=True


Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,054,704 parameters, 0 gradients, 67.8 GFLOPs

image 1/51 /content/ToothNumber_TaskDataset/images/test/0ba65172-20240821-105924223.jpg: 640x640 1 Canine (13), 1 Canine (23), 1 Canine (33), 1 Canine (43), 1 Central Incisor (21), 1 Central Incisor (41), 1 Central Incisor (31), 2 Central Incisor (11)s, 1 First Molar (16), 1 First Molar (26), 1 First Molar (36), 1 First Molar (46), 1 First Premolar (14), 1 First Premolar (34), 1 First Premolar (44), 1 First Premolar (24), 1 Lateral Incisor (22), 1 Lateral Incisor (32), 2 Lateral Incisor (42)s, 1 Lateral Incisor (12), 1 Second Molar (17), 1 Second Molar (27), 1 Second Molar (37), 1 Second Molar (47), 1 Second Premolar (15), 1 Second Premolar (25), 1 Second Premolar (35), 1 Second Premolar (45), 1 Third Molar (18), 1 Third Molar (28), 1 Third Molar (38), 2 Third Molar (48)s, 37.0ms
image 2/51 /content/ToothNumber_TaskData

In [15]:
from ultralytics import YOLO
import numpy as np
import os
import pandas as pd


In [17]:
model = YOLO("runs/detect/train/weights/best.pt")


In [18]:
results = model.predict(
    source="/content/ToothNumber_TaskDataset/images/test",
    save=False,
    conf=0.25
)



image 1/51 /content/ToothNumber_TaskDataset/images/test/0ba65172-20240821-105924223.jpg: 640x640 1 Canine (13), 1 Canine (23), 1 Canine (33), 1 Canine (43), 1 Central Incisor (21), 1 Central Incisor (41), 1 Central Incisor (31), 2 Central Incisor (11)s, 1 First Molar (16), 1 First Molar (26), 1 First Molar (36), 1 First Molar (46), 1 First Premolar (14), 1 First Premolar (34), 1 First Premolar (44), 1 First Premolar (24), 1 Lateral Incisor (22), 1 Lateral Incisor (32), 2 Lateral Incisor (42)s, 1 Lateral Incisor (12), 1 Second Molar (17), 1 Second Molar (27), 1 Second Molar (37), 1 Second Molar (47), 1 Second Premolar (15), 1 Second Premolar (25), 1 Second Premolar (35), 1 Second Premolar (45), 1 Third Molar (18), 1 Third Molar (28), 1 Third Molar (38), 2 Third Molar (48)s, 37.0ms
image 2/51 /content/ToothNumber_TaskDataset/images/test/18fd6ab8-20250415-143542497.jpg: 640x640 1 Canine (13), 1 Canine (23), 1 Canine (33), 1 Canine (43), 1 Central Incisor (21), 1 Central Incisor (41), 1 C

In [19]:
def post_process_predictions(predictions, iou_thresh=0.5):
    final_preds = []
    for dets in predictions:
        cleaned = []
        used_classes = {}

        for box, conf, cls in zip(dets.boxes.xyxy, dets.boxes.conf, dets.boxes.cls):
            x1, y1, x2, y2 = box.tolist()
            cls = int(cls.item())
            conf = float(conf.item())
            det = [x1, y1, x2, y2, conf, cls]

            if cls not in used_classes:
                used_classes[cls] = det
            else:
                # IoU check
                ux1, uy1, ux2, uy2, _, _ = used_classes[cls]
                inter_x1 = max(x1, ux1)
                inter_y1 = max(y1, uy1)
                inter_x2 = min(x2, ux2)
                inter_y2 = min(y2, uy2)
                inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
                area1 = (x2-x1)*(y2-y1)
                area2 = (ux2-ux1)*(uy2-uy1)
                iou = inter_area / float(area1 + area2 - inter_area)

                if iou < iou_thresh:
                    cleaned.append(det)

        cleaned.extend(used_classes.values())
        final_preds.append(cleaned)

    return final_preds

cleaned_results = post_process_predictions(results)


In [20]:
def assign_fdi_numbers(detections, img_shape):
    H, W = img_shape
    mid_y = H / 2   # horizontal midline
    mid_x = W / 2   # vertical midline

    quadrants = {"UL": [], "UR": [], "LL": [], "LR": []}

    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        cx = (x1 + x2) / 2
        cy = (y1 + y2) / 2

        if cy < mid_y:  # upper arch
            if cx < mid_x:
                quadrants["UL"].append(det)
            else:
                quadrants["UR"].append(det)
        else:  # lower arch
            if cx < mid_x:
                quadrants["LL"].append(det)
            else:
                quadrants["LR"].append(det)

    # sort each quadrant left → right
    for key in quadrants:
        quadrants[key] = sorted(quadrants[key], key=lambda b: (b[0]+b[2])/2)

    # Assign FDI numbering
    fdi_map = {
        "UR": list(range(11, 19)),
        "UL": list(range(21, 29)),
        "LL": list(range(31, 39)),
        "LR": list(range(41, 49))
    }

    assigned = []
    for key in quadrants:
        teeth = quadrants[key]
        fdi_numbers = fdi_map[key][:len(teeth)]
        for det, fdi in zip(teeth, fdi_numbers):
            assigned.append({
                "fdi": fdi,
                "bbox": det[:4],
                "conf": det[4],
                "cls": det[5],
                "quadrant": key
            })

    return assigned


In [21]:
all_results = []

for dets, res in zip(cleaned_results, results):
    img_shape = res.orig_shape
    assigned_teeth = assign_fdi_numbers(dets, img_shape)

    for t in assigned_teeth:
        all_results.append({
            "image": os.path.basename(res.path),
            "fdi_number": t["fdi"],
            "class_id": t["cls"],
            "confidence": t["conf"],
            "quadrant": t["quadrant"],
            "bbox": t["bbox"]
        })


In [ ]:
df = pd.DataFrame(all_results)
df.to_csv("/content/final_tooth_predictions.csv", index=False)

print(" Final results saved: /content/final_tooth_predictions.csv")
